In [ ]:
import math
import tkinter as tk
from tkinter import ttk, messagebox
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class NanoparticleBuilder:
    def __init__(self, root):
        self.root = root
        self.root.title("Core-Shell Nanoparticle Builder")

        self.layers = []  # Store layers as (radius, comp)
        self.colorbar = None  # Store colorbar reference

        # UI for adding layers
        self.layer_frame = ttk.Frame(root)
        self.layer_frame.pack(padx=10, pady=10, fill="x")

        ttk.Label(self.layer_frame, text="Shell Radius (nm)").grid(row=0, column=0, padx=5)
        ttk.Label(self.layer_frame, text="Composition").grid(row=0, column=1, padx=5)

        self.radius_var = tk.DoubleVar()
        self.comp_var = tk.StringVar()

        self.radius_entry = ttk.Entry(self.layer_frame, textvariable=self.radius_var, width=10)
        self.radius_entry.grid(row=1, column=0, padx=5)

        self.comp_dropdown = ttk.Combobox(self.layer_frame, textvariable=self.comp_var, width=15)
        self.comp_dropdown["values"] = [
            "Ag_evap", "Pena26Au74Ag", "Pena100Ag", "Pena34Au66Ag",
            "Au_evap", "Pena15Au85Ag", "Pena52Au48Ag"
        ]
        self.comp_dropdown.grid(row=1, column=1, padx=5)

        ttk.Button(self.layer_frame, text="+ Add Layer", command=self.add_layer).grid(row=1, column=2, padx=5)
        ttk.Button(self.layer_frame, text="- Remove Last Layer", command=self.remove_layer).grid(row=1, column=3, padx=5)

        # Display layers
        self.layer_list = tk.Listbox(root, height=8)
        self.layer_list.pack(padx=10, pady=5, fill="x")

        # Plot area
        self.figure, self.ax = plt.subplots(figsize=(5, 4))
        self.canvas = FigureCanvasTkAgg(self.figure, master=root)
        self.canvas_widget = self.canvas.get_tk_widget()
        self.canvas_widget.pack(padx=10, pady=5)

        # Plot and Save buttons
        self.button_frame = ttk.Frame(root)
        self.button_frame.pack(pady=10)

        ttk.Button(self.button_frame, text="Plot Structure", command=self.plot_structure).pack(side="left", padx=5)
        ttk.Button(self.button_frame, text="Save Shape and Parameter File", command=self.save_files).pack(side="right", padx=5)

    def add_layer(self):
        radius = self.radius_var.get()
        comp = self.comp_var.get()
        if radius <= 0 or not comp:
            messagebox.showerror("Input Error", "Radius must be positive and Composition must be selected.")
            return

        if self.layers and radius <= self.layers[-1][0]:
            messagebox.showerror("Input Error", "Radius must be larger than the previous layer.")
            return

        self.layers.append((radius, comp))
        self.layer_list.insert(tk.END, f"Radius: {radius} nm, Composition: {comp}")
        self.radius_var.set(0)
        self.comp_var.set("")

    def remove_layer(self):
        if self.layers:
            self.layers.pop()
            self.layer_list.delete(tk.END)

    def plot_structure(self):
        if not self.layers:
            messagebox.showerror("Error", "No layers to plot.")
            return

        # Clear and recreate the figure and axes to ensure a clean plot
        self.figure.clf()  # Clear the entire figure
        self.ax = self.figure.add_subplot(111)  # Recreate the axes

        # Generate a 2D slice through the center of the nanoparticle
        N = 256
        L = 0.160  # microns
        d = L / N

        x = np.linspace(-L / 2, L / 2, N)
        y = np.linspace(-L / 2, L / 2, N)
        X, Y = np.meshgrid(x, y)
        r = np.sqrt(X**2 + Y**2) * 1e3  # Convert to nm

        # Reset the composition grid and compute new values
        composition_grid = np.zeros_like(r, dtype=int)
        for i, (layer_radius, comp) in enumerate(self.layers, start=1):
            composition_grid[(r <= layer_radius) & (composition_grid == 0)] = i

        # Plot the new composition grid
        cax = self.ax.imshow(
            composition_grid,
            cmap="viridis",
            origin="lower",
            extent=[-L / 2 * 1e3, L / 2 * 1e3, -L / 2 * 1e3, L / 2 * 1e3],
        )
        self.ax.set_title("2D Cross-Section of Nanoparticle")
        self.ax.set_xlabel("X (nm)")
        self.ax.set_ylabel("Y (nm)")

        # Add a new colorbar
        self.figure.colorbar(cax, ax=self.ax, label="Composition")

        # Redraw the canvas
        self.canvas.draw()

    def save_files(self):
        if not self.layers:
            messagebox.showerror("Error", "No layers to save.")
            return

        # Save shape.dat
        shape_data = []

        # Define lattice spacing (adjust these values as needed)
        dx = 0.002  # Spacing in micrometers
        dy = 0.002
        dz = 0.002

        # Compute the number of dipoles based on lattice spacing
        L = 0.160  # Total size in micrometers
        Nx = math.ceil(L / dx) + 1
        Ny = math.ceil(L / dy) + 1
        Nz = math.ceil(L / dz) + 1

        shape_data.append(">TARGSPHER  spherical target with explicit spacing")
        shape_data.append(f"{Nx * Ny * Nz} = NAT")
        shape_data.append("1.000000  0.000000  0.000000 = A_1 vector")
        shape_data.append("0.000000  1.000000  0.000000 = A_2 vector")
        shape_data.append("1.000000  1.000000  1.000000 = (d_x,d_y,d_z)/d")
        shape_data.append(f"-{Nx//2}.5 -{Ny//2}.5 -{Nz//2}.5 = lattice offset x0(1-3) = (x_TF,y_TF,z_TF)/d for dipole 1,1,1")
        shape_data.append("    JA   IX   IY   IZ ICOMP(x,y,z)")

        JA = 1  # Dipole counter

        # Sort layers by radius to ensure inner radius is processed first
        sorted_layers = sorted(self.layers, key=lambda x: x[0])

        for ix in range(Nx):
            for iy in range(Ny):
                for iz in range(Nz):
                    # Compute spatial position
                    x = (ix - (Nx - 1) / 2) * dx
                    y = (iy - (Ny - 1) / 2) * dy
                    z = (iz - (Nz - 1) / 2) * dz
                    r = np.sqrt(x**2 + y**2 + z**2) * 1e3  # Convert to nm

                    # Determine composition from the innermost layer outward
                    comp = 0
                    for i, (layer_radius, _) in enumerate(sorted_layers, start=1):
                        if r <= layer_radius:
                            comp = i
                            break

                    # Add only non-zero composition dipoles
                    if comp != 0:
                        shape_data.append(f"{JA:6d} {ix+1:3d} {iy+1:3d} {iz+1:3d} {comp:1d} {comp:1d} {comp:1d}")
                        JA += 1

        # Update the total number of dipoles in the header
        shape_data[1] = f"{JA - 1} = NAT"

        with open("shape.dat", "w") as f:
            f.write("\n".join(shape_data))

        # Save ddscat.par
        param_data = [
            "' ========== Parameter file for v7.3 ==================='",
            "'**** Preliminaries ****'",
            "'NOTORQ' = CMTORQ*6 (DOTORQ, NOTORQ) -- either do or skip torque calculations",
            "'PBCGS2' = CMDSOL*6 (PBCGS2, PBCGST, GPBICG, QMRCCG, PETRKP) -- CCG method",
            "'GPFAFT' = CMETHD*6 (GPFAFT, FFTMKL) -- FFT method",
            "'GKDLDR' = CALPHA*6 (GKDLDR, LATTDR, FLTRCD) -- DDA method",
            "'NOTBIN' = CBINFLAG (NOTBIN, ORIBIN, ALLBIN)",
            "'**** Initial Memory Allocation ****'",
            "100 100 100 = dimensioning allowance for target generation",
            "'**** Target Geometry and Composition ****'",
            "'FROM_FILE' = CSHAPE*9 shape directive",
            "no SHPAR parameters needed",
            f"{len(self.layers)}         = NCOMP = number of dielectric materials",
        ]

        for i, (_, comp) in enumerate(self.layers, start=1):
            param_data.append(f"'../diel/{comp}' = file with refractive index {i}")

        param_data.extend([
            "'**** Additional Nearfield calculation? ****'",
            "0 = NRFLD (=0 to skip nearfield calc., =1 to calculate nearfield E)",
            "0.0 0.0 0.0 0.0 0.0 0.0 (fract. extens. of calc. vol. in -x,+x,-y,+y,-z,+z)",
            "'**** Error Tolerance ****'",
            "1.00e-7 = TOL = MAX ALLOWED (NORM OF |G>=AC|E>-ACA|X>)/(NORM OF AC|E>)",
            "'**** Maximum number of iterations ****'",
            "10000     = MXITER",
            "'**** Integration cutoff parameter for PBC calculations ****'",
            "1.00e-2 = GAMMA (1e-2 is normal, 3e-3 for greater accuracy)",
            "'**** Angular resolution for calculation of <cos>, etc. ****'",
            "0.5\t= ETASCA (number of angles is proportional to [(3+x)/ETASCA]^2 )",
            "'**** Vacuum wavelengths (micron) ****'",
            "0.500 0.500 1 'INV' = wavelengths (first,last,how many,how=LIN,INV,LOG)",
            "'**** Refractive index of ambient medium'",
            "1.333 = NAMBIENT",
            "'**** Effective Radii (micron) **** '",
            "0.246186 0.246186 1 'LIN' = eff. radii (first, last, how many, how=LIN,INV,LOG)",
            "'**** Define Incident Polarizations ****'",
            "(0,0) (1.,0.) (0.,0.) = Polarization state e01 (k along x axis)",
            "2 = IORTH  (=1 to do only pol. state e01; =2 to also do orth. pol. state)",
            "'**** Specify which output files to write ****'",
            "1 = IWRKSC (=0 to suppress, =1 to write .sca file for each target orient.",
            "'**** Specify Target Rotations ****'",
            "0.    0.   1  = BETAMI, BETAMX, NBETA  (beta=rotation around a1)",
            "0.    0.   1  = THETMI, THETMX, NTHETA (theta=angle between a1 and k)",
            "0.    0.   1  = PHIMIN, PHIMAX, NPHI (phi=rotation angle of a1 around k)",
            "'**** Specify first IWAV, IRAD, IORI (normally 0 0 0) ****'",
            "0   0   0    = first IWAV, first IRAD, first IORI (0 0 0 to begin fresh)",
            "'**** Select Elements of S_ij Matrix to Print ****'",
            "6\t= NSMELTS = number of elements of S_ij to print (not more than 9)",
            "11 12 21 22 31 41\t= indices ij of elements to print",
            "'**** Specify Scattered Directions ****'",
            "'LFRAME' = CMDFRM (LFRAME, TFRAME for Lab Frame or Target Frame)",
            "1 = NPLANES = number of scattering planes",
            "0.  0. 180. 5 = phi, theta_min, theta_max (deg) for plane A",
            "90. 0. 180. 5 = phi, theta_min, theta_max (deg) for plane B"
        ])

        with open("ddscat.par", "w") as f:
            f.write("\n".join(param_data))

        messagebox.showinfo("Success", "Files saved as shape.dat and ddscat.par!")

if __name__ == "__main__":
    root = tk.Tk()
    app = NanoparticleBuilder(root)
    root.mainloop()
